## PID using the beam monitors
This code is the default way of identifying particles using the beam monitor. It is providing only a template event selection that is not optimised for any Physics analyses. It should serve as an exmple around which to build your own beam particle identification code. 

In [1]:
#Step 0, import libraries
import numpy as np
import importlib
#this is the file with the necessary functions for performing PID 
import beam_monitors_pid as bm


In [2]:
#Step 1, read in the data 

#### Example 1: medium momentum negative polarity
# run_number = 1478
# run_momentum = -410
# n_eveto_group = 1.01 #refractive index of ACT0-2
# n_tagger_group = 1.06 # of ACT3-5
# there_is_ACT5 = False  #important to keep track of whether there is ACT5 in the run  


### Example 2: relatively high momentum, positive polarity
run_number = 1610
run_momentum = 760
n_eveto_group = 1.01
n_tagger_group = 1.015
there_is_ACT5 = True

##### Example 2.5: relatively high momentum, positive polarity
# run_number = 1602
# run_momentum = 770
# n_eveto_group = 1.01
# n_tagger_group = 1.015
# there_is_ACT5 = True

######## Example 2.5.1: relatively high momentum positive polarity
# run_number = 1606
# run_momentum = 780
# n_eveto_group = 1.01
# n_tagger_group = 1.015
# there_is_ACT5 = True


###Example 3: low momentum, positive polarity 
# run_number = 1308
# run_momentum = 220
# n_eveto_group = 1.01
# n_tagger_group = 1.15
# there_is_ACT5 = False

#choose the number of events to read in, set to -1 if you want to read all events
n_events = 100000 #-1

#Set up a beam analysis class 
ana = bm.BeamAnalysis(run_number, run_momentum, n_eveto_group, n_tagger_group, there_is_ACT5)

#Store into memory the number of events desired
ana.open_file(n_events)

Reading in events: 100%|██████████| 100000/100000 [00:15<00:00, 6361.67evt/s]

Total weight of self.df: 19.72Mb


In [3]:
#Step 2: Adjust the 1pe calibration: need to check the accuracy on the plots
# which are stored in plots/PID_run{run_number}_p{run_momentum}.pdf
ana.adjust_1pe_calibration()

One PE calibration finished, please don't forget to check that it is correct


In [4]:
#Step 3: proton and heavier particle tagging with T0-T1 TOF
#We need to tag protons before any other particles to avoid double-counting
ana.tag_protons_TOF()
#TODO: identify protons that produce knock-on electrons 

A total of 37733 protons and 261 deuterons nuclei are tagged using the TOF out of 98958, i.e. 38.1% of the dataset are protons and 0.3% are deuteron
A total of 25 helium3 nuclei, 14 tritium nuclei and 9 lithium 6 nuclei are tagged using the TOF out of 98958, i.e. 0.03% of the dataset are helium3, 0.0% are tritium, 0.01 lithium 6 nuclei


In [5]:
#Step 4: tag electrons using ACT0-2 finding the minimum in the cut line
#If we want a tighter cut, add a coefficient of reduction of the optimal cut line (e.g. 5%) to remove more electrons (and also some more muons and pions) 
tightening_factor = 0 #in units of percent of the cut line, how much you want to reduce the cut position to increase the purity of the muon/pion sample
#this is interseting but not really resolving the issue of electron contamination: leave at 0% for now
ana.tag_electrons_ACT02(tightening_factor)

#instead use ACT35 to tag electrons (when depositing more than cutline PE, for now TBD by analyser)
cut_line = 30 #PE
ana.tag_electrons_ACT35(cut_line)


A total of 9391 electrons are tagged with ACT02 out of 98958, i.e. 9.5% of the dataset
ACT35 left vs right plots have been made please check that they are sensible, electrons should be in the top right corner
ACT35 left vs right plots have been made please check that they are sensible, protons should be in the bottom left corner
A total of 142 additional electrons are tagged with ACT35, on top of the 9391 that were tagged with ACT02


In [6]:
#Step 5: check visually that the electron and proton removal makes sense in ACT35
ana.plot_ACT35_left_vs_right()

ACT35 left vs right plots have been made please check that they are sensible, electrons should be in the top right corner
ACT35 left vs right plots have been made please check that they are sensible, protons should be in the bottom left corner


In [7]:
#Step 6: make the muon/pion separation, using the muon tagger in case 
#at least 0.5% of muons and pions are above the cut line. This is necessary in case the 
#Number of particles is too high to clearly see a minimum between the muons and pions
#A more thorough analysis might want to remove events that are close to the cut line for a higher purity
ana.tag_muons_pions_ACT35()


The muon tagger charge has been plotted. The optimal cut line is at 185.3 a.u., there are 439 electrons above the cut line and 404 muons and pions, i.e. 0.8% of all muons and pions (51383)...
there are more than 0.5% of muons and pions above the muon tagger cut, we are applying it. (Please verify this on the plots)
The muon scale (at 12.1), is 8.6
We assume that all pions with more than 19.4 PE in ACT35 are actually electrons


/eos/home-i01/a/acraplet/analysis_tools/notebooks/beam_monitors_pid.py:1193: RuntimeWarning: invalid value encountered in divide
  ratio = h_muon_scaled/(h_muon_scaled+h_pion+h_electron_scaled)
/eos/home-i01/a/acraplet/analysis_tools/notebooks/beam_monitors_pid.py:1195: RuntimeWarning: invalid value encountered in divide
  ratio2 = h_pion/(h_muon_scaled+h_pion+h_electron_scaled)
/eos/home-i01/a/acraplet/analysis_tools/notebooks/beam_monitors_pid.py:1198: RuntimeWarning: invalid value encountered in divide
  ratio_pi_mu = h_electron_scaled/(h_muon_scaled+h_pion+h_electron_scaled)


The sum of L_mu is 1.00


/eos/home-i01/a/acraplet/analysis_tools/notebooks/beam_monitors_pid.py:1257: RuntimeWarning: divide by zero encountered in log
  ax2.step(bin_centers, np.log(L_mu+eps) - np.log(L_pi+eps), color = "green", where='mid', label=f"log(L_mu) - log(L_pi)")
/eos/home-i01/a/acraplet/analysis_tools/notebooks/beam_monitors_pid.py:1257: RuntimeWarning: invalid value encountered in log
  ax2.step(bin_centers, np.log(L_mu+eps) - np.log(L_pi+eps), color = "green", where='mid', label=f"log(L_mu) - log(L_pi)")
/eos/home-i01/a/acraplet/analysis_tools/notebooks/beam_monitors_pid.py:1257: RuntimeWarning: invalid value encountered in subtract
  ax2.step(bin_centers, np.log(L_mu+eps) - np.log(L_pi+eps), color = "green", where='mid', label=f"log(L_mu) - log(L_pi)")
/eos/home-i01/a/acraplet/analysis_tools/notebooks/beam_monitors_pid.py:1258: RuntimeWarning: divide by zero encountered in log
  ax2.step(bin_centers, np.log(L_pi+eps) - np.log(L_e+eps), color = "magenta", where='mid', label=f"log(L_pi) - log(L_e)

ACT35 left vs right plots have been made please check that they are sensible, electrons should be in the top right corner
ACT35 left vs right plots have been made please check that they are sensible, protons should be in the bottom left corner
The pion/muon separtion cut line in ACT35 is 3.6, out of 51383.0 pions and muons, 9.0% are muons and 91.0 are pions


In [8]:
#Step 7: estimate the momentum for each particle from the T0-T1 TOF
#Note: we will save:
# 1. the momentum as the particle escapes the beam pipe and its error
# 2. the momentum as the particle escapes the WCTE beam window and its error
# 3. the mean momentum for this particle type and the associated error 

# first measure the particle TOF, make the plot
#This corrects any offset in the TOF (e.g. from cable length) that can cause the TOF 
#of electrons to be different from L/c This has to be calibrated to give meaningful momentum 
#estimates later on
ana.measure_particle_TOF()


The time difference between the reconstructed electron TOF and L/c = 14.81 is 0.86 ns


In [9]:
#This function extimates both the mean momentum for each particle type and for each trigger
#We take the the error on the tof for each trigger is the resolution of the TS0-TS1 measurement
#Taken as the std of the gaussian fit to the electron TOF
#This is still a somewhat coarse way of estimating uncertainty... 
#This also saves the momentum after exiting the beam window, recosntructed using the same techinque
#Final momentum is after exiting through the beam pipe

ana.estimate_momentum(-1.012e-2, True)


#note, because of fluctuations in the TOF, the reconstructed momentum will be unphysical for 
#some fo the events on the faster side of the distribution, this means that the 
#distibution of momenta event by event will be non-symetrical with a tail at low momenta
#The mean momentum will be fine though 


 The initial momenta considered for the piPlus are [ 228.          263.57446809  299.14893617  334.72340426  370.29787234
  405.87234043  441.44680851  477.0212766   512.59574468  548.17021277
  583.74468085  619.31914894  654.89361702  690.46808511  726.04255319
  761.61702128  797.19148936  832.76595745  868.34042553  903.91489362
  939.4893617   975.06382979 1010.63829787 1046.21276596 1081.78723404
 1117.36170213 1152.93617021 1188.5106383  1224.08510638 1259.65957447
 1295.23404255 1330.80851064 1366.38297872 1401.95744681 1437.53191489
 1473.10638298 1508.68085106 1544.25531915 1579.82978723 1615.40425532
 1650.9787234  1686.55319149 1722.12765957 1757.70212766 1793.27659574
 1828.85106383 1864.42553191 1900.        ]
After crossing the mylar beam pipe window, the momentum is [ 227.99991781  263.57439131  299.1488629   334.72333325  370.29780295
  405.87227195  441.44674075  477.02120937  512.59567789  548.17014613
  583.74461448  619.31908261  654.89355071  690.46801877  726.04

After ACT5, the Pions momentum is [ 218.8028691   254.98925706  290.92930006  326.74524704  362.46503671
  398.15588384  433.79886589  469.42318345  505.03688659  540.60987439
  576.19728651  611.77023656  647.33515268  682.89777271  718.45496626
  754.01560014  789.57124554  825.12633164  860.6798425   896.23478862
  931.78574126  967.34267145 1002.90120808 1038.43462437 1073.98925837
 1109.55421627 1145.10425316 1180.65234215 1216.19348849 1251.74741083
 1287.31153415 1322.87351065 1358.41079859 1393.97875298 1429.52227305
 1465.08067324 1500.64517135 1536.19075072 1571.74968643 1607.30674409
 1642.86489149 1678.42189878 1713.97988393 1749.5245026  1785.09710392
 1820.65693472 1856.21710428 1891.77690965] and the TOF is [16.80518364 16.194978   15.78017145 15.48568214 15.2694041  15.10590929
 14.97943996 14.87965232 14.79954253 14.73429461 14.68043945 14.63549327
 14.59760319 14.56534992 14.5376881  14.51378032 14.49298138 14.47477349
 14.45874513 14.44456189 14.43195267 14.42069154 

/eos/home-i01/a/acraplet/analysis_tools/notebooks/beam_monitors_pid.py:1626: RuntimeWarning: invalid value encountered in sqrt
  momentum[i] = np.sqrt((particle_kinetic_energy + masses[particle_name])**2 - masses[particle_name]**2)


The estimated mean particle momenta are {'electron': 0, 'muon': np.float64(647.1725967893346), 'pion': np.float64(744.5441649515734), 'proton': np.float64(748.1272030949966), 'deuteron': np.float64(737.2495956644543), 'helium3': np.float64(1681.7896493614007), 'tritium': 0, 'lithium6': 0} MeV/c with an error {'electron': 0, 'muon': np.float64(4.483773914539256), 'pion': np.float64(1.2078652197594693), 'proton': np.float64(0.08967142321057509), 'deuteron': np.float64(1.1081318032727268), 'helium3': np.float64(10.003619852220481), 'tritium': 0, 'lithium6': 0} MeV/c


/eos/home-i01/a/acraplet/analysis_tools/notebooks/beam_monitors_pid.py:1626: RuntimeWarning: invalid value encountered in sqrt
  momentum[i] = np.sqrt((particle_kinetic_energy + masses[particle_name])**2 - masses[particle_name]**2)
/eos/home-i01/a/acraplet/analysis_tools/notebooks/beam_monitors_pid.py:1626: RuntimeWarning: invalid value encountered in sqrt
  momentum[i] = np.sqrt((particle_kinetic_energy + masses[particle_name])**2 - masses[particle_name]**2)
/eos/home-i01/a/acraplet/analysis_tools/notebooks/beam_monitors_pid.py:2035: RuntimeWarning: invalid value encountered in divide
  intercept = (y_b * x_a - y_a * x_b) / (x_a - x_b)
/eos/home-i01/a/acraplet/analysis_tools/notebooks/beam_monitors_pid.py:2035: RuntimeWarning: invalid value encountered in divide
  intercept = (y_b * x_a - y_a * x_b) / (x_a - x_b)
/eos/home-i01/a/acraplet/analysis_tools/notebooks/beam_monitors_pid.py:2035: RuntimeWarning: invalid value encountered in divide
  intercept = (y_b * x_a - y_a * x_b) / (x_a 


 
      act0_l    act1_l    act2_l    act3_l    act4_l    act5_l    act0_r  \
0  0.005403  0.040733  0.023963  0.035790  0.048029  0.074270  0.118077   
1 -0.016728  0.002572 -0.028942 -0.005662  0.621316 -0.026957  0.039810   
2 -0.060989 -0.006968  0.004124  0.004701 -0.047519 -0.007977  0.000676   
3 -0.038858  0.002572 -0.042169 -0.036751  0.012199  0.023657  0.078943   
4  0.027533  0.012112  0.030576  0.009882  0.878100 -0.014303  0.176778   

     act1_r    act2_r    act3_r  ...      L_mu      L_pi       L_e  is_muon  \
0  0.823921  0.010049  0.065466  ...  0.012821  0.141107  0.003933    False   
1 -0.044104 -0.024895  1.193726  ...  0.000000  0.000000  0.000000    False   
2  0.004120  0.038005  0.041563  ...  0.012821  0.141107  0.003933    False   
3 -0.024814  0.010049  0.003317  ...       NaN       NaN       NaN    False   
4  0.033054 -0.017907  0.003317  ...  0.015385  0.375445  0.008292    False   

   is_pion   tof_corr  initial_momentum  initial_momentum_error  \
0  

In [10]:
#Visually, it looks like all the particles reach the TOF
ana.plot_TOF_charge_distribution()


In [11]:
#Step X: end_analysis, necessary to cleanly close files 
ana.end_analysis()

In [12]:
#Output to a root file
ana.output_beam_ana_to_root()

/eos/home-i01/a/acraplet/analysis_tools/notebooks/beam_monitors_pid.py:3060: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[False  True False ...  True  True False]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.df_all.loc[self.df_all["is_kept"], col] = self.df[col].values
/eos/home-i01/a/acraplet/analysis_tools/notebooks/beam_monitors_pid.py:3060: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[False False False ... False False False]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.df_all.loc[self.df_all["is_kept"], col] = self.df[col].values
/eos/home-i01/a/acraplet/analysis_tools/notebooks/beam_monitors_pid.py:3060: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future versio

Saved output file to beam_analysis_output_R1610.root


### List of the relevant beam PID information saved to the output file
After the beam PID has been performed we are saving the relevant variables to an output file, ideally root, to compare with other reconstructions:

##### Branch: beam_analysis

1. ACT i right (charge in ACT i right PMT, in units of PE) 
2. ACT i left  (for precise selections)
3. ACT02 total (for coarse selections)
4. ACT35 total 
5. T0-T1 TOF (called "tof")
6. T0-T4 TOF #this still has bugs I think
7. T4-T1 TOF  #this still has bugs I think
8. Muon tagger information (left and right, and total)

16. Estimated PID from the beam information #todo, have a likelihood for each particle type
10. Estimated initial momenta for each trigger and error on mean
10. Estimated momenta exiting the beampipe for each trigger and error
17. Whether event passes beam data quality cuts
18. Total TOF detector charge, one can add a selection cut to remove events which do not cross the TOF for further analysis
19. ref0 and ref1 times (reference times of each digitiser) 

##### Branch: run_info
14. run number
13. run momentum (nominal, from CERN)
14. Aerogel refrective index information
15. Whether ACT5 is in the beamline

##### Branch: scalar_results

18. Position of each cut line and whether we apply the muon tagger cut for pion/muon separation (see step 6)
9. Mean tof for each particle type and error on mean
9. Mean momenta for each particle type, gaussian std and error on mean



#### This is the end of the analysis please check the plots on plots/PID_run{run_number}\_p{run_momentum}.pdf and the analysis output in beam_analysis_output_R{run_number}.root